In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import random
from sklearn.model_selection import train_test_split

2023-05-06 15:44:09.574550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 15:44:09.681062: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 15:44:09.683752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/don/anaconda3/envs/cab420/lib/python3.10/site-packages/cv2/../../lib64:
2023-

In [2]:
num_classes=23
image_height=800
image_width=1200

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (image_width, image_height))
    x = x/255.0
    x = x.astype(np.float32)
    return x


def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (image_width, image_height))
    x = x.astype(np.int32)
    return x


def tf_dataset(x,y, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((x,y)) # Dataset object from Tensorflow
    dataset = dataset.shuffle(buffer_size=100) 
    dataset = dataset.map(preprocess) # Applying preprocessing to every batch in the Dataset object
    dataset = dataset.batch(batch) # Determine atch-size
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2) # Optimization
    return dataset
        

def preprocess(x,y):
    def f(x,y):
        x = x.decode()
        y = y.decode()
        image = read_image(x)
        mask = read_mask(y)
        return image, mask
    
    image, mask = tf.numpy_function(f,[x,y],[tf.float32, tf.int32])
    mask = tf.one_hot(mask, num_classes, dtype=tf.int32)
    image.set_shape([image_height, image_width, 3])    # In the Images, number of channels = 3. 
    mask.set_shape([image_height, image_width, num_classes])    # In the Masks, number of channels = number of classes. 
    return image, mask

In [3]:
root_dir = '/home/don/Git/aerial-semantic-segmentation/dataset_here/dataset/semantic_drone_dataset'
img_path = root_dir + '/original_images/'
mask_path = root_dir + '/label_images_semantic/'

names = list(map(lambda x: x.replace('.jpg', ''), os.listdir(img_path)))


In [4]:
X_trainval, X_test = train_test_split(names, test_size=0.1, random_state=19)
X_train, X_val = train_test_split(X_trainval, test_size=0.2, random_state=19)

print(f"Train Size : {len(X_train)} images")
print(f"Val Size   :  {len(X_val)} images")
print(f"Test Size  :  {len(X_test)} images")

y_train = X_train
y_test = X_test
y_val = X_val

img_train = [os.path.join(img_path, f"{name}.jpg") for name in X_train]
mask_train = [os.path.join(mask_path, f"{name}.png") for name in y_train]
img_val = [os.path.join(img_path, f"{name}.jpg") for name in X_val]
mask_val = [os.path.join(mask_path, f"{name}.png") for name in y_val]
img_test = [os.path.join(img_path, f"{name}.jpg") for name in X_test]
mask_test = [os.path.join(mask_path, f"{name}.png") for name in y_test]

Train Size : 288 images
Val Size   :  72 images
Test Size  :  40 images


In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize and train the SVM classifier
svm = SVC()
svm.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = svm.predict(X_val)

# Calculate the accuracy of the SVM on the validation set
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the validation accuracy
print("Validation Accuracy:", val_accuracy)


ValueError: Expected 2D array, got 1D array instead:
array=[ 77. 121. 568. 223.  31. 497.   3.   5. 320. 433.  55. 260. 141.  38.
 216. 175. 136. 451. 208.  43. 323. 582. 335. 122. 138. 304. 325. 311.
 165. 480. 104. 312. 383. 408. 301. 110. 475. 177. 455. 228.  58. 163.
 180. 431. 148. 314. 273. 412. 302. 390. 188. 234. 592. 419. 161.  57.
 538.  89. 576. 318. 199. 590. 341. 128.  41. 288. 113. 554. 159. 443.
 294. 154.  18. 515. 217. 119.  52. 580. 101. 345.  16.  99. 149. 445.
 591. 501. 292. 134. 330. 222. 545. 265. 173. 381.   6. 130. 414. 193.
 263. 598. 289. 107. 543. 213. 225. 190. 351. 146. 276. 170. 452. 251.
 349. 570. 535. 298. 200. 410. 464. 434. 421. 457. 157. 321. 305. 435.
 232.  44. 126.  62.   1. 147.  88. 139. 425. 382. 484. 584. 426. 332.
 233.  51. 167. 389. 156. 427. 366. 587. 313. 235. 220. 521.  78. 494.
 380. 361. 192. 176. 498. 424. 329. 299. 281. 403.  11. 463. 226.  56.
 499. 488. 583.  47. 240. 458. 430. 117. 439. 194. 275. 559. 533. 508.
  71. 493.  19. 52